In [1]:
!pip3 install transformers
!pip3 install SentencePiece
import numpy
from transformers import T5Tokenizer, T5ForConditionalGeneration
import csv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.1 MB/s eta 0:00:00


In [2]:
model_name = "t5-large"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use th

In [3]:
def question_to_template(question, demonstrations):
    # A refined prompt, making the instruction more explicit
    input_text = f"{demonstrations} {question}"
    # input_text = f"{demonstrations}+ {question}"

    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(input_ids)

    # Extract the generated template
    template = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return template

# demos = 'What is the size of the TV? The size of the TV is [mask]. What is on the TV screen? [mask] is on the TV screen. What is the giraffe standing behind? The giraffe is standing behind [mask]. What does the wall say on the far left? The wall on the far left say [mask]. Which direction is the turn lane going? The lane is going [mask]. What color hat is this man wearing? The man is wearing [mask]. What is causing the smoke? The cause of the smoke is [mask]. What is your name? Your name is [mask]. Where is the Eiffel Tower? The Eiffel Tower is in [mask]. What color is the locomotive? The color of the locomotive is [mask].What kind of animal is on the beach? The animal on the beach is [mask]  '

# question = "How many striped objects are in the photo? <extra_id_0>"
# template = question_to_template(question,demos)
# print(template)

In [4]:
import json
with open('/content/drive/MyDrive/Colab Notebooks/v2_OpenEnded_mscoco_train2014_questions.json', 'r') as file:
    annotations_data = json.load(file)

In [9]:
len(list(annotations_data['questions']))

443757

In [5]:
with open('questions_and_templates.csv', 'a', newline='') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(["Question", "Template"])

  for item in annotations_data['questions']:
    question = item['question']
    if 'what' in question[:4].lower() or 'where' in question[:5].lower():
      demos = 'What is the size of the TV? The size of the TV is [mask]. What is on the TV screen? [mask] is on the TV screen. What is the giraffe standing behind? The giraffe is standing behind [mask]. What does the wall say on the far left? The wall on the far left say [mask].What color hat is this man wearing? The man is wearing [mask]. What is causing the smoke? The cause of the smoke is [mask]. What is your name? Your name is [mask]. Where is the Eiffel Tower? The Eiffel Tower is in [mask]. What color is the locomotive? The color of the locomotive is [mask].What kind of animal is on the beach? The animal on the beach is [mask]  '
    elif 'which' in question[:5].lower():
      demos = "Which hot dog has a larger variety of toppings? The [mask] has a larger variety of toppings.Which operating system is being used on this computer? The [mask] operating system is being used on this computer. Which side of the room is the television probably on? The [mask] side of the room is the television probably on. Which utensil is on the table in the foreground? The [mask] utensil is on the table in the foreground.Which way is the train going? The [mask] way is the train going. Which hand holds the racket? The [mask] hand holds the racket. Which foot is lifted in the air? The [mask] foot is lifted in the air "
    elif 'how many' in question[:9].lower() or 'how old' in question[:9].lower() or 'how old' in question[:9].lower():
      demos = "How many unopened rolls of paper are in the picture? There are [mask] unopened rolls of paper in the picture. How many engines does the closest airplane have? The closest airplane has [mask] engines. How many different types of doors are visible? There are [mask] different types of doors visible. How many people are wearing plaid shirts? There are [mask] people wearing plaid shirts.How many people are participating in the eating contest? There are [mask] people participating in the eating contest. How many cabinets have been installed? There are [mask] cabinets installed."
    elif any(word in question[:5].lower() for word in ['does','is','are','do','will','could','did','can','would','has','have','was','has']):
      demos = "Does this food look burnt? This food looks burnt.[mask]. Does this appear to be a noisy environment? This appears to be a noisy environment. Is the sky blue? The sky is blue Does this type of fruit change color? This type of fruit does not change color.Does this animal produce dairy products? This animal does not produce dairy products."
    elif "why" in question[:4].lower():
      demos = "Why are the birds only appearing as black silhouettes? The birds are only appearing as black silhouettes because [mask]. Why is the man sad? The man is sad because [mask]. Why is everyone afraid? Everyone is afraid because [mask]. Why is the woman running? The woman is running because [mask]. Why is the food warm? The food is warm because [mask]"
    elif 'who' in question[:4].lower() or 'whose' in question[:5].lower():
      demos = "Who made the cock? [mask] made the cock. Who is crying? [mask] is crying. Who is hiding behind the curtain? [mask] is hiding behind the curtain. Who is the man talking to? The man is talking to [mask]. Who is going to the bathroom? [mask] is going to the bathroom. Who is sitting on the sofa? [mask] is sitting on the sofa. Who is climbing the laddeR? [mask] is climbing the ladder"

    else:
      writer.writerow([question, "<BLANK>"])
      pass
    question = question + " <extra_id_0>"
    template = question_to_template(question,demos)
    # print(question + '--->' + template)
    writer.writerow([question, template])
  print('Completed')


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


KeyboardInterrupt: ignored

In [22]:
def get_n_best_completions(template, n=20):

    input_text = template.replace("[mask]", "<extra_id_0>")
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    outputs = model.generate(input_ids, num_return_sequences=5, top_k=5, num_beams=n, temperature=0.1)

    completions = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outputs]

    return completions



template = "The child is doing [mask]."
print(get_n_best_completions(template))

['well........', '........', 'well.......', 'well......', 'great........']


In [11]:
# def get_filtered_answers(question_template, answer_vocabulary, k=5):
#     # Get the likelihood of each answer fitting into the template
#     likelihoods = {}
#     for answer in answer_vocabulary:
#         prompt = question_template.replace("[mask]", answer)
#         input_tensor = tokenizer(prompt, return_tensors="pt").input_ids
#         output = model.generate(input_tensor)
#         decoded_output = tokenizer.decode(output[0])

#         # A simple method: the likelihood is based on how close the model's output is to the input
#         likelihoods[answer] = sum(1 for a, b in zip(decoded_output, prompt) if a == b)

#     # Get top k answers based on likelihood
#     top_k_answers = sorted(likelihoods, key=likelihoods.get, reverse=True)[:k]

#     return top_k_answers

In [ ]:
# question_template = "The species of the flower is [mask]"
# answer_template = ["rose","tulip","daisy","vase","table"]
# top_answers = get_filtered_answers(question_template, answer_template,k=4)

In [ ]:
top_answers

['rose', 'tulip', 'daisy', 'vase']

In [ ]:
https://visualqa.org/download.html

In [ ]:
#Second method
!pip3 install stanza

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.2/881.2 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 36.0 MB/s eta 0:00:00


In [ ]:
import stanza

# Initialize the English model
nlp = stanza.Pipeline('en')

def question_to_template_with_parsing(question):
    # Parse the question
    doc = nlp(question)

    # Extract important words and their relations
    wh_word = None
    root_word = None
    aux_word = None
    subject_word = None
    object_word = None

    for word in doc.sentences[0].words:
      print(word,word.deprel)

question = "Does this room have a low ceiling?"
template = question_to_template_with_parsing(question)
print(template)


INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| pos          | combined_charlm     |
| lemma        | combined_nocharlm   |
| constituency | ptb3-revised_charlm |
| depparse     | combined_charlm     |
| sentiment    | sstplus             |
| ner          | ontonotes_charlm    |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


{
  "id": 1,
  "text": "Does",
  "lemma": "do",
  "upos": "AUX",
  "xpos": "VBZ",
  "feats": "Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin",
  "head": 4,
  "deprel": "aux",
  "start_char": 0,
  "end_char": 4
} aux
{
  "id": 2,
  "text": "this",
  "lemma": "this",
  "upos": "DET",
  "xpos": "DT",
  "feats": "Number=Sing|PronType=Dem",
  "head": 3,
  "deprel": "det",
  "start_char": 5,
  "end_char": 9
} det
{
  "id": 3,
  "text": "room",
  "lemma": "room",
  "upos": "NOUN",
  "xpos": "NN",
  "feats": "Number=Sing",
  "head": 4,
  "deprel": "nsubj",
  "start_char": 10,
  "end_char": 14
} nsubj
{
  "id": 4,
  "text": "have",
  "lemma": "have",
  "upos": "VERB",
  "xpos": "VB",
  "feats": "VerbForm=Inf",
  "head": 0,
  "deprel": "root",
  "start_char": 15,
  "end_char": 19
} root
{
  "id": 5,
  "text": "a",
  "lemma": "a",
  "upos": "DET",
  "xpos": "DT",
  "feats": "Definite=Ind|PronType=Art",
  "head": 7,
  "deprel": "det",
  "start_char": 20,
  "end_char": 21
} det
{
  "id": 6,
